In [1]:
pip install SimpleITK

     |████████████████████████████████| 48.4 MB 32 kB/s 


In [2]:
from pathlib import Path
from multiprocessing import Pool
import logging

import click
import pandas as pd
import numpy as np
import SimpleITK as sitk



In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [6]:
# Default paths

path_in = '/gdrive/MyDrive/HECKTOR Dataset/original_hecktor/hecktor2021_train/hecktor_nii/'
# path_in = '/gdrive/MyDrive/Head _Neck Hector/CleanData_nii_Registerated/'
path_out = '/gdrive/MyDrive/Tecvico-Datasets/Datasets/HN_Cancer/Head _Neck Hector/NewHector_final/'
path_bb = '/gdrive/MyDrive/Tecvico-Datasets/Datasets/HN_Cancer/Head _Neck Hector/bbox.csv'


In [11]:
def main(input_folder, output_folder, bounding_boxes_file):
    """ This command line interface allows to resample NIFTI files within a
        given bounding box contain in BOUNDING_BOXES_FILE. The images are
        resampled with spline interpolation
        of degree 3 and the segmentation are resampled
        by nearest neighbor interpolation.

        INPUT_FOLDER is the path of the folder containing the NIFTI to
        resample.
        OUTPUT_FOLDER is the path of the folder where to store the
        resampled NIFTI files.
        BOUNDING_BOXES_FILE is the path of the .csv file containing the
        bounding boxes of each patient.
    """


    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    output_folder.mkdir(exist_ok=True)

    bb_df = pd.read_csv(bounding_boxes_file)
    bb_df = bb_df.set_index('PatientID')
    patient_list = [f.name.split("_")[0] for f in input_folder.rglob("*_ct*")]
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputDirection([1, 0, 0, 0, 1, 0, 0, 0, 1])


    def resample_one_patient(p):
        bb = np.array([
            bb_df.loc[p, 'x1'], bb_df.loc[p, 'y1'], bb_df.loc[p, 'z1'],
            bb_df.loc[p, 'x2'], bb_df.loc[p, 'y2'], bb_df.loc[p, 'z2']
        ])
        ct = sitk.ReadImage(
            str([f for f in input_folder.rglob(p + "_ct*")][0].resolve()))
        pt = sitk.ReadImage(
            str([f for f in input_folder.rglob(p + "_pt*")][0].resolve()))
        gtvt = sitk.ReadImage(
            str([f for f in input_folder.rglob(p + "_gtvt*")][0].resolve()))
        resampling = ct.GetSpacing()
        size = np.round((bb[3:] - bb[:3]) / resampling).astype(int)

        resampler.SetOutputSpacing(resampling)
        resampler.SetOutputOrigin(bb[:3])
        resampler.SetSize([int(k) for k in size])  # sitk is so stupid
        resampler.SetInterpolator(sitk.sitkBSpline)
        ct = resampler.Execute(ct)
        pt = resampler.Execute(pt)
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        gtvt = resampler.Execute(gtvt)

        output_folder1 = output_folder / "ct"
        sitk.WriteImage(ct, str(
            (output_folder1 / (p + "_ct.nii.gz")).resolve()))
        
        output_folder1 = output_folder / "pet"
        sitk.WriteImage(pt, str(
            (output_folder1 / (p + "_pt.nii.gz")).resolve()))
        
        output_folder1 = output_folder / "gtv"
        sitk.WriteImage(gtvt,
                        str((output_folder1 / (p + "_gtvt.nii.gz")).resolve()))
    iii = 0
    for p in patient_list:
        resample_one_patient(p)
        iii = iii + 1
        print(iii)
        # if iii == 3:
        #   break


In [ ]:
main(path_in ,path_out ,path_bb  )

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
